## Install Dependencies

In [3]:
!pip install -U "transformers==4.40.0" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 21.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2


In [4]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using c

In [5]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.2 MB/s eta 0:00:00


In [6]:
from huggingface_hub import login
token = "YOUR_TOKEN_HERE"
login(token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Download Gemma 7b-it

In [7]:
import transformers
import torch

model_id = "google/gemma-7b-it"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

## Query Gemma 7b-it

In [2]:
messages = [
    {"role"   : "user",
     "content": """
     Identify the types of fallacies present in the text I present to you.

The output should be formatted as follows: [{"text": "", "class": "", "fallacy": ""}]
If multiple fallacies occur separate them with comma's
"text", here you need to quote the sentence in which the fallacy occurs.
"class", here you need to give the number of the fallacy category, only write the number that is written before the category name. The categories are shown below.
"fallacy", here you need to give only name of the fallacy that occurs, from the list below.

1) Fallacy of Credibility: Abusive; Ad Hominem; Ad Populum; Appeal to False Authority; Appeal to Nature; Appeal to Tradition; Guilt by Association; Tu Quoque
2) Fallacy of Logic: Causal Oversimplification; Circular Reasoning; Equivocation; False Analogy; False Causality; False Dilemma; Hasty Generalization; Slippery Slope; Straw Man; Fallacy of Division
3) Appeal to Emotion: Appeal to Positive Emotion; Appeal to Anger; Appeal to Fear; Appeal to Pity; Appeal to Ridicule; Appeal to Worse Problem


Here is the text:
"America is the best place to live, because it's better than any other country."
Only output the list of fallacies, nothing else. Make sure that the fallacy is within the options that I gave to you.

     """},

]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print(outputs[0]["generated_text"][len(prompt):])

NameError: name 'pipeline' is not defined

## Generate knowledge prompting

In [15]:
messages = [
    {"role"   : "user",
     "content": """

      First generate an explanation of the following fallacies:
      1) Fallacy of Credibility: Abusive; Ad Hominem; Ad Populum; Appeal to False Authority; Appeal to Nature; Appeal to Tradition; Guilt by Association; Tu Quoque
      2) Fallacy of Logic: Causal Oversimplification; Circular Reasoning; Equivocation; False Analogy; False Causality; False Dilemma; Hasty Generalization; Slippery Slope; Straw Man; Fallacy of Division
      3) Appeal to Emotion: Appeal to Positive Emotion; Appeal to Anger; Appeal to Fear; Appeal to Pity; Appeal to Ridicule; Appeal to Worse Problem

      Second identify which of the fallacies you just explained, occur in the text below:
      "America is the best place to live, because it's better than any other country."

      The output for the identification of the fallacy should be formatted as follows: [{"text": "", "class": "", "fallacy": ""}]
      If multiple fallacies occur separate them with comma's
      "text", here you need to quote the sentence in which the fallacy occurs.
      "class", here you need to give the number of the fallacy category, only write the number that is written before the category name. The categories are shown below.
      "fallacy", here you need to give only name of the fallacy that occurs, from the list below.


     """},

]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print(outputs[0]["generated_text"][len(prompt):])

**Fallacies Explanation:**

**1) Fallacy of Credibility:**
- Abusive: Personal attacks on the credibility of the speaker or source.
- Ad Hominem: Attacking the character of the opponent rather than their argument.
- Ad Populum: Appeal to popularity or authority figures.
- Appeal to False Authority: Using the authority of a person or organization that is not credible.
- Appeal to Nature: Invoking natural phenomena to support an argument.
- Appeal to Tradition: Relying on tradition or custom without questioning.
- Guilt by Association: Associating someone with something negative to cast doubt on their character.
- Tu Quoque: Attacking the speaker's own credibility to undermine their argument.

**2) Fallacy of Logic:**
- Causal Oversimplification: Oversimplifying the causes of a problem to make it easier to understand.
- Circular Reasoning: Relying on the same premise to prove itself.
- Equivocation: Using a fallacy to disguise the true meaning of an argument.
- False Analogy: Drawing an 